In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
import datasets

In [3]:
datasets.load_trn("iris")

/home/yutongw/UCI_classification/jupyter_notebooks


{'data': array([[-8.97674e-01,  1.02861e+00, -1.33679e+00, -1.30859e+00],
        [-1.38073e+00,  3.36720e-01, -1.39347e+00, -1.30859e+00],
        [-1.50149e+00,  1.06090e-01, -1.28012e+00, -1.30859e+00],
        [-5.35384e-01,  1.95113e+00, -1.16677e+00, -1.04652e+00],
        [-1.50149e+00,  7.97981e-01, -1.33679e+00, -1.17756e+00],
        [-1.01844e+00,  7.97981e-01, -1.28012e+00, -1.30859e+00],
        [-1.74302e+00, -3.55171e-01, -1.33679e+00, -1.30859e+00],
        [-1.13920e+00,  1.06090e-01, -1.28012e+00, -1.43963e+00],
        [-5.35384e-01,  1.48987e+00, -1.28012e+00, -1.30859e+00],
        [-1.25996e+00,  7.97981e-01, -1.22344e+00, -1.30859e+00],
        [-1.25996e+00, -1.24540e-01, -1.33679e+00, -1.43963e+00],
        [-1.86378e+00, -1.24540e-01, -1.50682e+00, -1.43963e+00],
        [-5.35384e-01,  1.95113e+00, -1.39347e+00, -1.04652e+00],
        [-1.73094e-01,  1.72050e+00, -1.16677e+00, -1.17756e+00],
        [-8.97674e-01,  1.72050e+00, -1.28012e+00, -1.17756e+00],
  

In [4]:
import numpy as np
import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import tensorflow as tf

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import losses
import metrics
import utils

from wwsvm import WWSVM

ModuleNotFoundError: No module named 'losses'